In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

from skimage.io import imread, imshow
from skimage.transform import resize

import tensorflow as tf
from tensorflow.python.keras.backend import set_session
from tensorflow.keras import layers, Model


import os
import random
import numpy as np

from tqdm import tqdm

from PIL import Image
from sklearn.metrics import accuracy_score
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
## Variaveis
seeds = [13, 42]
np.random.seed(0)
num_classes = 1
split_size = 0.2
k = 5
IMG_HEIGHT = 128
IMG_WIDTH = 128
IMG_CHANNELS = 1
NUM_TEST_IMAGES = 10


# Definindo o caminho da pasta
imagens = 'Imagens1'
mascara = 'mascaras1'

# Listando todos os arquivos na pasta
imagens_lista = os.listdir(imagens)
mascaras_lista = os.listdir(mascara)

# Inicializando listas para cada coluna
imagens_id = []
mascara_id = []


In [ ]:
# Loop pelos arquivos e preenchendo as listas com informações da pasta de imagens:
for arquivo in imagens_lista:
    # Nome e extensão do arquivo
    imagens_id.append(arquivo)

# Criando o DataFrame das imagens
df_img = pd.DataFrame({
    'Imagens_id': imagens_id
})

# Loop pelos arquivos e preenchendo as listas com informações da pasta de mascaras:
for arquivo in mascaras_lista:
    # Nome e extensão do arquivo
    mascara_id.append(arquivo)

# Criando o DataFrame das mascaras
df_msk = pd.DataFrame({
    'mascaras_id': mascara_id
})

In [ ]:
print(df_img.head(10))
print(df_img.shape)

print(df_msk.head(10))
print(df_msk.shape)

In [ ]:
# Get lists of images and their masks.

image_id_list = list(df_img['Imagens_id'])
mask_id_list = list(df_msk['mascaras_id'])

# Create empty arrays

X = np.zeros((len(image_id_list), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

y = np.zeros((len(mask_id_list), IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)

X_test = np.zeros((NUM_TEST_IMAGES, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

In [ ]:
# X imagens

for i, imagens_id in enumerate(image_id_list):

    path_image = 'Imagens1/' + imagens_id

    # read the image using skimage
    image = imread(path_image)

    # resize the image
    image = resize(image, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

    # use np.expand dims to add a channel axis so the shape becomes (IMG_HEIGHT, IMG_WIDTH, 1)
    image = np.expand_dims(image, axis=-1)

    # insert the image into X_train
    X[i] = image

print(X.shape)

In [ ]:
# Y classes (mascras)


for i, mascara_id in enumerate(mask_id_list):

    path_mask = 'mascaras1/' + mascara_id

    # read the image using skimage
    mask = imread(path_mask)
    mask = (mask >= 250)

    # resize the image
    mask = resize(mask, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

    # use np.expand dims to add a channel axis so the shape becomes (IMG_HEIGHT, IMG_WIDTH, 1)
    mask = np.expand_dims(mask, axis=-1)

    # insert the image into Y_Train
    y[i] = mask

print(y.shape)

In [ ]:
# Sample image and Ground-Truth Label from dataset

image_x = random.randint(0, len(X))
plt.axis("off")
imshow(X[image_x])
plt.show()
plt.axis("off")
imshow(np.squeeze(y[image_x]))
plt.show()

In [ ]:
!pip install keras-unet-collection
from keras_unet_collection import models


In [ ]:
acc = []
jacc = []
f1 = []
prec = []
rec = []


for f in range(1, len(seeds)):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=split_size, random_state=seeds[f])


    model = models.unet_plus_2d((128, 128, 1), [16, 32, 64, 128], n_labels=1,
                            stack_num_down=2, stack_num_up=2,
                            activation='LeakyReLU', output_activation='Sigmoid',
                            batch_norm=False, pool='max', unpool=False, deep_supervision=True, name='xnet')

    # Define metrics for each output
    metrics = ['accuracy', tf.keras.metrics.Recall(name='recall')]
    metrics_dict = {output_name: metrics for output_name in model.output_names}

    model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Use a single loss function for the final output
    metrics= metrics_dict
     )
    checkpoint_filepath = 'model1_' + str(f+1)+'fold.keras'
    callbacks = [
            tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss'),
            tf.keras.callbacks.TensorBoard(log_dir='logs'),
            tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_filepath,
                save_weights_only=False,
                monitor='val_recall',
                mode='max',
                save_best_only=True,
                verbose=1)]

    model.fit(X_train, y_train, validation_data=(X_val,y_val), batch_size=16, epochs=5, callbacks=callbacks)

    for i in range(0, len(X_val)):
        sample_image = X_val[i]
        sample_mask = y_val[i].astype(np.uint8).flatten()
        prediction = model.predict(sample_image[tf.newaxis, ...],verbose=0)[0]
        predicted_mask = (prediction > 0.5).astype(np.uint8).flatten()

        acc.append(accuracy_score(sample_mask, predicted_mask))
        jacc.append(jaccard_score(sample_mask, predicted_mask))
        f1.append(f1_score(sample_mask, predicted_mask))
        prec.append(precision_score(sample_mask, predicted_mask))
        rec.append(recall_score(sample_mask, predicted_mask))

    del model


In [ ]:
predicted_mask.shape